In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_columns', None)

In [2]:
# 1월 
data1 = pd.read_csv("C:/Users/ellyj/Desktop/24-1/퀀트 예측/데이터/2023_2024_df.csv")
data1.shape

(34648614, 6)

In [3]:
# 변수 9개 추가 생성
def convert_tick_to_ohlcv(data):
    """
    Converts given Binance tick data into 1-hour interval OHLCV (Open, High, Low, Close, Volume) data.
    :param data: DataFrame with Tick data
    :return: DataFrame with the Open, High, Low, Close, Volume values
    """

    data['time'] = pd.to_datetime(data['time'], unit='ms')
    ohlcv = data.resample('1H', on='time').agg({
        'price': ['first', 'max', 'min', 'last', "median", "mean"],
        'qty': ['sum', 'median'],
        'quote_qty': ['max', 'min', 'median', 'mean'],
        'id' : ['count'],
        'is_buyer_maker': ['sum']})

    ohlcv.columns = ['Open', 'High', 'Low', "Close", "median_price", "mean_price",
                     'Volume', 'median_qty',
                     "max_quote", "min_quote", "median_quote", "mean_qute",
                     "num_id",
                     "maker"]
    ohlcv['taker'] = ohlcv['num_id'] - ohlcv['maker']
    return ohlcv

In [5]:
df_ohlcv1 = convert_tick_to_ohlcv(data1)
df_ohlcv1 = df_ohlcv1.reset_index()
df_ohlcv1

,time,Open,High,Low,Close,median_price,mean_price,Volume,median_qty,max_quote,min_quote,median_quote,mean_qute,num_id,maker,taker
0,2024-01-01 00:00:00,42314.0,42603.2,42289.6,42503.5,42466.30,42462.482375,8459.477,0.014,3.131629e+06,42.290000,592.9154,4068.922552,88278,41778,46500
1,2024-01-01 01:00:00,42503.5,42832.0,42462.0,42647.9,42702.40,42665.175592,9043.411,0.016,1.281000e+06,42.464700,683.6944,4271.895931,90351,40965,49386
2,2024-01-01 02:00:00,42647.9,42676.9,42530.0,42620.4,42596.80,42600.060137,4653.067,0.015,8.518000e+05,42.532800,639.3780,3772.038846,52550,28065,24485
3,2024-01-01 03:00:00,42620.5,42630.0,42270.0,42369.8,42430.00,42431.296439,8119.880,0.012,8.770914e+05,42.270600,510.5628,4083.495819,84345,48188,36157
4,2024-01-01 04:00:00,42369.8,42439.8,42235.2,42436.6,42350.30,42345.917605,6356.536,0.012,2.606825e+06,42.238000,508.4202,3956.523507,68026,31386,36640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,2024-01-09 17:00:00,46901.2,46922.2,46623.0,46718.8,46770.10,46762.061397,10947.187,0.010,1.143394e+06,46.624100,467.6610,3202.897780,159824,80410,79414
210,2024-01-09 18:00:00,46718.7,46732.8,46500.0,46643.6,46606.30,46604.877172,10505.851,0.010,1.436259e+06,46.500000,465.9390,3114.250286,157219,80612,76607
211,2024-01-09 19:00:00,46643.5,46895.9,46570.7,46873.5,46755.10,46750.381553,7880.575,0.010,6.319347e+05,46.572600,467.2130,2863.686492,128653,61802,66851
212,2024-01-09 20:00:00,46873.5,46937.0,46585.4,46636.1,46745.40,46754.298905,8565.049,0.010,9.588251e+05,46.588700,467.2040,2895.039686,138307,68309,69998


In [6]:
df_ohlcv1["time"].max()

Timestamp('2024-01-09 21:00:00')

In [8]:
data2 = pd.read_csv("C:/Users/ellyj/Desktop/24-1/퀀트 예측/데이터/10_12data.csv")
print(data2.shape)
data2.head()

(2208, 32)


,time,Open,High,Low,Close,median_price,mean_price,Volume,median_qty,max_quote,min_quote,median_quote,mean_qute,num_id,maker,taker,Open_4,High_4,Low_4,Close_4,median_price_4,mean_price_4,Volume_4,median_qty_4,max_quote_4,min_quote_4,median_quote_4,mean_qute_4,num_id_4,maker_4,taker_4,previous_week_close
0,2023-10-01 00:00:00,26951.0,26986.6,26939.0,26976.9,26961.3,26961.452439,2733.691,0.005,724874.150,26.9390,134.73075,2035.624480,36206,14354,21852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
1,2023-10-01 01:00:00,26976.9,27026.8,26963.8,27005.1,27000.6,26996.394031,3161.822,0.003,599054.790,26.9638,81.02940,2221.352144,38429,18832,19597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2,2023-10-01 02:00:00,27005.1,27037.5,27004.4,27015.6,27021.6,27021.129339,2681.754,0.005,540500.000,27.0044,135.10050,2020.383251,35867,18967,16900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
3,2023-10-01 03:00:00,27015.7,27039.7,26993.6,27033.0,27021.6,27020.477518,2415.426,0.004,327018.201,26.9936,108.15600,2308.719073,28271,14114,14157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
4,2023-10-01 04:00:00,27032.9,27035.0,27004.2,27035.0,27019.5,27020.768965,1889.918,0.005,405300.000,27.0042,135.09325,2015.088402,25342,11238,14104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [10]:
# 12월 데이터
data2['time'] = pd.to_datetime(data2['time'])  
data2 = data2[data2['time'].dt.month == 12]

In [15]:
data2.columns

Index(['time', 'Open', 'High', 'Low', 'Close', 'median_price', 'mean_price',
       'Volume', 'median_qty', 'max_quote', 'min_quote', 'median_quote',
       'mean_qute', 'num_id', 'maker', 'taker', 'Open_4', 'High_4', 'Low_4',
       'Close_4', 'median_price_4', 'mean_price_4', 'Volume_4', 'median_qty_4',
       'max_quote_4', 'min_quote_4', 'median_quote_4', 'mean_qute_4',
       'num_id_4', 'maker_4', 'taker_4', 'previous_week_close'],
      dtype='object')

In [16]:
data2 = data2.loc[:, ['time', 'Open', 'High', 'Low', 'Close', 'median_price', 'mean_price',
       'Volume', 'median_qty', 'max_quote', 'min_quote', 'median_quote',
       'mean_qute', 'num_id', 'maker', 'taker']]
data2

,time,Open,High,Low,Close,median_price,mean_price,Volume,median_qty,max_quote,min_quote,median_quote,mean_qute,num_id,maker,taker
1464,2023-12-01 00:00:00,37716.9,37734.6,37600.0,37665.6,37640.0,37646.560339,6134.170,0.010,3.729732e+06,37.6000,376.70050,3839.935313,60132,31366,28766
1465,2023-12-01 01:00:00,37665.5,37960.0,37626.6,37927.3,37874.9,37846.185621,15632.157,0.014,9.761730e+05,37.6280,526.94600,3959.049723,149447,68325,81122
1466,2023-12-01 02:00:00,37927.4,38070.0,37852.1,37977.8,37936.0,37947.376001,10909.863,0.013,2.589661e+06,37.8540,492.85990,3951.579651,104799,50360,54439
1467,2023-12-01 03:00:00,37977.9,38056.8,37933.4,38030.2,37994.9,37993.881136,8535.380,0.012,1.788774e+06,37.9335,456.55920,3690.627241,87869,45774,42095
1468,2023-12-01 04:00:00,38030.1,38236.6,38005.9,38170.5,38158.0,38142.838166,16655.059,0.014,1.775737e+06,38.0066,534.79860,4008.735791,158474,77707,80767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,2023-12-31 19:00:00,42701.7,42741.9,42624.7,42659.9,42679.6,42679.290527,3944.096,0.012,1.231618e+06,42.6247,512.53800,3764.214080,44718,23305,21413
2204,2023-12-31 20:00:00,42659.9,42724.5,42543.3,42599.1,42619.6,42625.853604,4730.936,0.013,9.464693e+05,42.5499,554.18480,3593.632603,56117,28014,28103
2205,2023-12-31 21:00:00,42599.2,42717.0,42558.2,42558.9,42632.9,42634.523521,3794.010,0.012,7.796380e+05,42.5604,510.84360,3637.087286,44473,21732,22741
2206,2023-12-31 22:00:00,42559.0,42629.5,42111.9,42294.8,42400.0,42405.779788,11952.346,0.015,2.079024e+06,42.1119,632.10225,4204.106744,120512,67159,53353


In [13]:
# 8,9,10,11 
data3 = pd.read_csv("C:/Users/ellyj/Desktop/24-1/퀀트 예측/데이터/891011월.csv")
data3

,time,Open,High,Low,Close,median_price,mean_price,Volume,median_qty,max_quote,min_quote,median_quote,mean_qute,num_id,maker,taker
0,2023-08-01 00:00:00,29220.8,29277.0,29200.9,29272.3,29236.6,29236.868213,4934.266,0.005,527177.420,29.2009,146.2830,2927.314022,49285,21048,28237
1,2023-08-01 01:00:00,29272.3,29337.3,29155.3,29185.7,29259.2,29256.025320,12283.595,0.008,822714.700,29.1553,233.3416,3213.154588,111857,61817,50040
2,2023-08-01 02:00:00,29185.7,29207.2,28684.7,28917.4,28937.5,28930.203801,49867.433,0.013,2885010.000,28.6847,376.3760,4395.999785,328085,188688,139397
3,2023-08-01 03:00:00,28917.4,28942.2,28800.0,28820.1,28881.6,28877.864573,21381.654,0.010,2566497.040,28.8000,288.3680,3939.735086,156707,81196,75511
4,2023-08-01 04:00:00,28820.1,28917.4,28785.3,28852.2,28853.4,28851.641791,15579.996,0.010,1872554.497,28.7853,288.3420,3947.813984,113858,51380,62478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2923,2023-11-30 19:00:00,37733.6,37794.3,37672.7,37745.3,37738.5,37738.940819,3651.796,0.010,943810.000,37.6745,376.9630,2845.269448,48436,24192,24244
2924,2023-11-30 20:00:00,37745.4,37756.0,37683.6,37720.1,37726.1,37725.163189,3098.473,0.009,378692.802,37.6845,339.6816,2449.435623,47719,24285,23434
2925,2023-11-30 21:00:00,37720.2,37760.3,37684.2,37730.8,37728.0,37726.242104,2299.085,0.008,278215.680,37.6846,301.6800,2265.821023,38281,19626,18655
2926,2023-11-30 22:00:00,37730.9,37805.0,37667.0,37685.8,37769.5,37758.239888,3399.703,0.010,1888250.000,37.6671,377.7770,2946.610060,43564,23665,19899


In [ ]:
df = pd.concat([data3, data2, df_ohlcv1], axis = 0, ignore_index= True)
print(df.shape)
df.head()

In [ ]:
df.to_csv("C:/Users/ellyj/Desktop/24-1/퀀트 예측/데이터/finaldata.csv", index = False)